In [17]:
import os
from dotenv import load_dotenv, find_dotenv
from pprint import pprint, pformat
import fitz  # Import the PyMuPDF library
import csv
import pandas as pd
from langchain import hub
from langchain_openai  import ChatOpenAI, OpenAIEmbeddings
from langchain.schema import AIMessage, HumanMessage, SystemMessage, Document
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

import tiktoken

from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent, create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI, OpenAI

from nanonets import NANONETSOCR


load_dotenv(find_dotenv())

True

### Global variables

In [18]:
input_file = r"N:\Dev\AI\Underwriting\data\clients\Safia Seyed\Safia Seyed Paystub.pdf"
# Extracted tables:
extracted_csvs_path = os.path.join(os.path.dirname(input_file), os.path.splitext(os.path.basename(input_file))[0] + " extracted_tables")
os.makedirs(extracted_csvs_path, exist_ok=True)
nano_extracted_tables_csv_path = os.path.join(extracted_csvs_path, os.path.splitext(os.path.basename(input_file))[0] + "_extracted_tables.csv" )

# Nanonets OCR

In [19]:
# model = NANONETSOCR()
# model.set_token(os.getenv("NANONESTS_TABLE_MODEL_TOKEN"))

In [20]:
# model.convert_to_csv(input_file, output_file_name=nano_extracted_tables_csv_path)

# Parse CSV

In [21]:
def extract_tables_to_csv(csv_file_path, output_path):
    # Extract directory and base name without extension
    csv_directory = os.path.dirname(csv_file_path)
    csv_base_name = os.path.basename(csv_file_path)
    csv_name_without_ext = os.path.splitext(csv_base_name)[0]

    # Create output folder path
    output_folder = output_path#os.path.join(csv_directory, csv_name_without_ext)

    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Initialize variables
    current_table_number = 0
    current_table_lines = []
    table_started = False
    table_paths = []  # Initialize an empty list to store file paths

    # Read the CSV file
    with open(csv_file_path, 'r', newline='') as file:
        reader = csv.reader(file)
        for row in reader:
            # Convert row list to string to check if it's a table header
            row_str = ','.join(row)
            if row_str.startswith('TABLE '):
                # If we're already in a table, save the current table
                if table_started:
                    path = save_table(current_table_lines, current_table_number, output_folder)
                    table_paths.append(path)  # Add the path to the list
                    current_table_lines = []  # Reset for the next table
                current_table_number += 1
                table_started = True  # Start a new table
            elif table_started:
                # Add row to current table
                current_table_lines.append(row)

    # Save the last table after file ends
    if table_started:
        save_table(current_table_lines, current_table_number, output_folder)
        table_paths.append(path)  # Add the path to the list
        
    return table_paths

def save_table(table_lines, table_number, output_folder):
    output_file_path = os.path.join(output_folder, f"Table_{table_number}.csv")

    # Check if the first row is empty or contains only commas
    if not any(table_lines[0]) or all(cell == '' for cell in table_lines[0]):
        table_lines = table_lines[1:]  # Remove the first row if it's empty

    with open(output_file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(table_lines)
    print(f"Table {table_number} saved to {output_file_path}")
    return output_file_path  # Return the path to the saved file

def extract_tables_to_dfs(csv_file_path):
    # Initialize variables
    current_table_number = 0
    current_table_lines = []
    table_started = False
    dfs = []  # Initialize an empty list to store DataFrames

    # Read the CSV file
    with open(csv_file_path, 'r', newline='') as file:
        reader = csv.reader(file)
        for row in reader:
            # Check if it's a table header
            row_str = ','.join(row)
            if row_str.startswith('TABLE '):
                # If we're already in a table, convert the current table to a DataFrame
                if table_started:
                    df = pd.DataFrame(current_table_lines[1:], columns=current_table_lines[0])
                    dfs.append(df)
                    current_table_lines = []  # Reset for the next table
                current_table_number += 1
                table_started = True  # Start a new table
            elif table_started:
                # Add row to current table
                current_table_lines.append(row)

    # Convert the last table to a DataFrame after file ends, if any
    if table_started:
        df = pd.DataFrame(current_table_lines[1:], columns=current_table_lines[0])
        dfs.append(df)

    return dfs

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ''
    for page in doc:
        text += page.get_text()
    doc.close()

    return text

In [32]:
# Example usage
# csv_file_path = 'data/pdfs/Yash Paystub Nov 17 2023_extracted.csv'
# csv_file_path = r'N:\Dev\AI\Underwriting\output3.csv'
table_paths = extract_tables_to_csv(nano_extracted_tables_csv_path, extracted_csvs_path)
table_dfs = extract_tables_to_dfs(nano_extracted_tables_csv_path)
table_text = extract_text_from_pdf(input_file)

Table 1 saved to N:\Dev\AI\Underwriting\data\clients\Safia Seyed\Safia Seyed Paystub extracted_tables\Table_1.csv
Table 2 saved to N:\Dev\AI\Underwriting\data\clients\Safia Seyed\Safia Seyed Paystub extracted_tables\Table_2.csv
Table 3 saved to N:\Dev\AI\Underwriting\data\clients\Safia Seyed\Safia Seyed Paystub extracted_tables\Table_3.csv
Table 4 saved to N:\Dev\AI\Underwriting\data\clients\Safia Seyed\Safia Seyed Paystub extracted_tables\Table_4.csv
Table 5 saved to N:\Dev\AI\Underwriting\data\clients\Safia Seyed\Safia Seyed Paystub extracted_tables\Table_5.csv
Table 6 saved to N:\Dev\AI\Underwriting\data\clients\Safia Seyed\Safia Seyed Paystub extracted_tables\Table_6.csv


In [23]:
print(table_paths)

['N:\\Dev\\AI\\Underwriting\\data\\clients\\Safia Seyed\\Safia Seyed Paystub extracted_tables\\Table_1.csv', 'N:\\Dev\\AI\\Underwriting\\data\\clients\\Safia Seyed\\Safia Seyed Paystub extracted_tables\\Table_2.csv', 'N:\\Dev\\AI\\Underwriting\\data\\clients\\Safia Seyed\\Safia Seyed Paystub extracted_tables\\Table_3.csv', 'N:\\Dev\\AI\\Underwriting\\data\\clients\\Safia Seyed\\Safia Seyed Paystub extracted_tables\\Table_4.csv', 'N:\\Dev\\AI\\Underwriting\\data\\clients\\Safia Seyed\\Safia Seyed Paystub extracted_tables\\Table_5.csv', 'N:\\Dev\\AI\\Underwriting\\data\\clients\\Safia Seyed\\Safia Seyed Paystub extracted_tables\\Table_5.csv']


In [24]:
table_dfs[0]

,,,,,,
0,,STATEMENT,OF\nBULLETIN DE,EARNINGS\nPAIE,,
1,TYPE,HOURS\nHEURES,RATE\nTAUX,AMOUNT\nMONTANT,Y.T.D.\nA JOUR,
2,SALARY,,,"2.591,88","15.551,28",
3,,,,,,
4,,,,,,


In [26]:
model = "gpt-4-turbo-preview"

dfs_chat_agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model=model), 
    table_dfs, 
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

csv_chat_agent = create_csv_agent(
    ChatOpenAI(temperature=0, model=model),
    table_paths,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

text_chat = ChatOpenAI(temperature=0, model=model)
prompt = f'''
The text below is extracted from financial payslips, answer the question based on the text provided.
Text:
f{table_text}

Question:

'''

In [11]:
question = "What is the base salary and what is the YTD earnings?"

In [41]:
csv_chat_agent.run(question)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df1'}`


  Unnamed: 0      STATEMENT OF\nBULLETIN DE   EARNINGS\nPAIE      Unnamed: 4  \
0       TYPE  HOURS\nHEURES      RATE\nTAUX  AMOUNT\nMONTANT  Y.T.D.\nA JOUR   
1     SALARY            NaN             NaN         2.591,88       15.551,28   
2        NaN            NaN             NaN              NaN             NaN   
3        NaN            NaN             NaN              NaN             NaN   

   Unnamed: 5  
0         NaN  
1         NaN  
2         NaN  
3         NaN  The base salary is 2,591.88, and the Year-To-Date (YTD) earnings are 15,551.28.

> Finished chain.


'The base salary is 2,591.88, and the Year-To-Date (YTD) earnings are 15,551.28.'

In [42]:
dfs_chat_agent.run(question)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df1.head()'}`


                                                                            
0              STATEMENT  OF\nBULLETIN DE   EARNINGS\nPAIE                  
1    TYPE  HOURS\nHEURES       RATE\nTAUX  AMOUNT\nMONTANT  Y.T.D.\nA JOUR  
2  SALARY                                         2.591,88       15.551,28  
3                                                                           
4                                                                           The base salary is 2,591.88, and the Year-To-Date (YTD) earnings are 15,551.28.

> Finished chain.


'The base salary is 2,591.88, and the Year-To-Date (YTD) earnings are 15,551.28.'

In [14]:
text_chat.invoke(prompt + question)

AIMessage(content='The base salary for the period is $2,591.88, and the Year-To-Date (YTD) earnings are $15,551.28.')

In [15]:
text_chat.invoke(prompt + "What is the start and end period of this payslip?")

AIMessage(content='The start and end period of this payslip are from 2023/10/01 to 2023/10/15.')

In [27]:
(dfs_chat_agent.agent.dict())

{'runnable': {'name': None,
  'first': {'name': None,
   'mapper': {'name': None,
    'steps': {'agent_scratchpad': RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))}}},
  'middle': [{'name': None,
    'input_variables': ['agent_scratchpad', 'input'],
    'input_types': {'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]},
    'output_parser': None,
    'partial_variables': {},
    'metadata': None,
    'tags': None,
    'messages': [{'content': "\nYou are working with 6 pandas dataframes in Python named df1, df2, etc.\nThis is the result of `print(df.head())` for each dataframe:\n|    |        |           |             |          |           |    |\n|---:|:-------|:----------|:------------|:------

# Custom agents

In [28]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Custom Tools

In [115]:
from langsmith import Client
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = f"Underwriting project"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
client = Client()

In [116]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [117]:
llm = ChatOpenAI(temperature=0, model=model)

from langchain.prompts import PromptTemplate

text_searcher_prompt = PromptTemplate.from_template('''Search the following text which was extracted from a PDF document and gather the information requested by the query. The text might be structured oddly so deductive reasoning might be needed to get the relevant information.  

Query:
{query}

Text:
{text}
'''
)

In [118]:
from typing import List


# Add error handling

@tool
def text_searcher(query: str) -> str:
    """Given a query the text_searcher tool will search the text to gather the information requested in the query"""
    
    return llm.invoke(text_searcher_prompt.format(query=query, text=table_text))

@tool
def csv_searcher(query: str) -> str:
    """Given a query the csv_searcher tool will use the csv_chat_agent to search the structured tables and gather the information requested in the query"""
    
    return csv_chat_agent.run(query)

tools = [text_searcher, csv_searcher]

In [119]:
from langchain_core.prompts import SystemMessagePromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, \
    ChatPromptTemplate
from pprint import pprint
agent_system_message = ''' You are an underwriting assistant who needs to extract and gather information from the provided documents. You must try to gather information from the structured CSV documents and varify them by searching the text directly. Some data might not be contained in the CSV files and should be gathered from the text.  
'''

agent_messages = [
    # SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=agent_system_message)),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')
]

temp = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=agent_system_message)),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)

temp
# 
# [temp.append(message) for message in agent_messages]
# temp

# from langchain import hub
# 
# # Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages[0].prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=' You are an underwriting assistant who needs to extract and gather information from the provided documents. You must try to gather information from the structured CSV documents

In [120]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [121]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [122]:
question = "What is the base salary and what is the YTD earnings?"
l = list(agent_executor.stream(
        {
            "input":question,
        }
    )
)



> Entering new AgentExecutor chain...

Invoking: `csv_searcher` with `{'query': 'base salary'}`




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df1['EARNINGS'][1]"}`


KeyError: 'EARNINGS'
Invoking: `python_repl_ast` with `{'query': 'df1.columns'}`


Index(['Unnamed: 0', 'STATEMENT', 'OF\nBULLETIN DE', 'EARNINGS\nPAIE',
       'Unnamed: 4', 'Unnamed: 5'],
      dtype='object')
Invoking: `python_repl_ast` with `{'query': "df1['EARNINGS\\nPAIE'][1]"}`


2.591,88The base salary is 2,591.88.

> Finished chain.
The base salary is 2,591.88.
Invoking: `csv_searcher` with `{'query': 'YTD earnings'}`




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df1['Unnamed: 4'][1]"}`


15.551,28The Year-To-Date (YTD) earnings are 15,551.28.

> Finished chain.
The Year-To-Date (YTD) earnings are 15,551.28.The base salary is $2,591.88, and the Year-To-Date (YTD) earnings are $15,551.28.

> Finished chain.


In [113]:
pprint(l)

[{'actions': [OpenAIToolAgentAction(tool='csv_searcher', tool_input={'query': 'base salary'}, log="\nInvoking: `csv_searcher` with `{'query': 'base salary'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_mR9zGxR9DpnhdIuRxNEJ29kt', 'function': {'arguments': '{"query": "base salary"}', 'name': 'csv_searcher'}, 'type': 'function'}, {'index': 1, 'id': 'call_H4HHIvChvdhMSvfA6ApD71Uq', 'function': {'arguments': '{"query": "YTD earnings"}', 'name': 'csv_searcher'}, 'type': 'function'}]})], tool_call_id='call_mR9zGxR9DpnhdIuRxNEJ29kt')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_mR9zGxR9DpnhdIuRxNEJ29kt', 'function': {'arguments': '{"query": "base salary"}', 'name': 'csv_searcher'}, 'type': 'function'}, {'index': 1, 'id': 'call_H4HHIvChvdhMSvfA6ApD71Uq', 'function': {'arguments': '{"query": "YTD earnings"}', 'name': 'csv_searcher'}, 'type': 'function'}]})]},
 {'actions': [Ope